In [1]:
from enum import Enum
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

/Users/artyom448/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('csv_result-dataset_50_tic-tac-toe.csv')

print(df.columns)
df.head

Index(['id', ''top-left-square'', ''top-middle-square'', ''top-right-square'',
       ''middle-left-square'', ''middle-middle-square'',
       ''middle-right-square'', ''bottom-left-square'',
       ''bottom-middle-square'', ''bottom-right-square'', ''Class''],
      dtype='object')
<bound method NDFrame.head of       id 'top-left-square' 'top-middle-square' 'top-right-square'  \
0      1                 x                   x                  x   
1      2                 x                   x                  x   
2      3                 x                   x                  x   
3      4                 x                   x                  x   
4      5                 x                   x                  x   
..   ...               ...                 ...                ...   
953  954                 o                   x                  x   
954  955                 o                   x                  o   
955  956                 o                   x                  o

In [3]:
rename_map={}
for col in df.columns:
    if col.startswith('\'') and col.endswith('\''):
        rename_map[col] = col[1:-1]
df = df.rename(columns=rename_map)

print(df.columns)

field_cols = ['top-left-square', 'top-middle-square', 'top-right-square',
              'middle-left-square', 'middle-middle-square',
              'middle-right-square', 'bottom-left-square',
              'bottom-middle-square', 'bottom-right-square']

df[field_cols] = df[field_cols].replace(['x', 'o', 'b'], [1, -1, 0])
df['Class'] = df.loc[:, 'Class'].replace(['positive', 'negative'], [1, -1])

print(df.head)

Index(['id', 'top-left-square', 'top-middle-square', 'top-right-square',
       'middle-left-square', 'middle-middle-square', 'middle-right-square',
       'bottom-left-square', 'bottom-middle-square', 'bottom-right-square',
       'Class'],
      dtype='object')
<bound method NDFrame.head of       id  top-left-square  top-middle-square  top-right-square  \
0      1                1                  1                 1   
1      2                1                  1                 1   
2      3                1                  1                 1   
3      4                1                  1                 1   
4      5                1                  1                 1   
..   ...              ...                ...               ...   
953  954               -1                  1                 1   
954  955               -1                  1                -1   
955  956               -1                  1                -1   
956  957               -1                  1  

In [4]:
train_size = 0.6
target = df['Class']
df.drop(columns=['Class'], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df, target, train_size=train_size, shuffle=True)

In [5]:
class ModelType(Enum):
    NearestNeighbours = 1
    SupportVectorMachine = 2
    BayessianClassifier = 3
    DecisionTree = 4
    AdaBoost = 5

def suggest_nearest_neighbours_model(trial):
    algorithm = trial.suggest_categorical('knn_algorithm', ['brute', 'kd_tree', 'ball_tree', 'auto'])
    leaf_size = trial.suggest_int('knn_leaf_size', 1, 200)

    return KNeighborsClassifier(
        algorithm=algorithm,
        n_neighbors=trial.suggest_int('knn_n_neighbors', 1, 400),
        weights=trial.suggest_categorical('knn_weights', ['uniform', 'distance']),
        p=trial.suggest_int('knn_p', 1, 3),
        leaf_size=leaf_size,
    )

def suggest_svm_model(trial):
    gamma = trial.suggest_categorical('svm_gamma_cat', ['scale', 'auto', 'float'])
    gamma = trial.suggest_float('svm_gamma_num', 0.1, 5.0) if gamma == 'float' else gamma
    kernel = trial.suggest_categorical('svm_kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
    coef0 = trial.suggest_float('svm_coef0', 0.1, 50.0) if kernel in ['poly', 'sigmoid'] else 0.0

    return SVC(
        degree=trial.suggest_int('svm_degree', 1, 5) if kernel == 'poly' else 1,
        shrinking=trial.suggest_categorical('svm_shrinking', [True, False]),
        probability=trial.suggest_categorical('svm_probability', [True, False]),
        tol=trial.suggest_float('svm_tol', 0.0001, 1.0),
        C=trial.suggest_float('svm_C', 0.1, 5.0),
        kernel=kernel,
        gamma=gamma,
        coef0=coef0,
        max_iter=500,
        random_state=1337,
    )

def suggest_baessian_classifier(trial):
    guess_priors = trial.suggest_categorical('bc_guess_priors', [True, False])
    priors = trial.suggest_float('bc_priors', 0.0, 1.0) if guess_priors else None
    priors = [priors, 1.0 - priors] if guess_priors else None

    return GaussianNB(
        var_smoothing=trial.suggest_float('bc_var_smoothing', 0.00000001, 0.01),
        priors=priors,
    )

def suggest_decision_tree(trial):
    limit_max_depth = trial.suggest_categorical('dt_limit_max_depth', [True, False])
    max_depth = trial.suggest_int('dt_max_depth', 5, 100) if limit_max_depth else None
    limit_max_leaf_nodes = trial.suggest_categorical('dt_limit_max_leaf_nodes', [True, False])
    max_leaf_nodes = trial.suggest_int('dt_max_leaf_nodes', 5, 300) if limit_max_leaf_nodes else None

    return DecisionTreeClassifier(
        splitter=trial.suggest_categorical('dt_splitter', ['best', 'random']),
        criterion=trial.suggest_categorical('dt_criterion', ['gini', 'entropy', 'log_loss']),
        min_samples_split=trial.suggest_int('dt_min_samples_split', 2, 50),
        min_samples_leaf=trial.suggest_int('dt_min_saples_leaf', 1, 50),
        max_features=trial.suggest_categorical('dt_max_features', ['sqrt', 'log2', None]),
        min_impurity_decrease=trial.suggest_float('dt_min_impurity_decrease', 0.0, 0.5),
        ccp_alpha=trial.suggest_float('dt_ccp_alpha', 0.0, 1.0), 
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_nodes,
        random_state=1337,
    )

def suggest_adaboost_model(trial):
    return AdaBoostClassifier(
        n_estimators=trial.suggest_int('ab_n_estimators', 5, 100),
        learning_rate=trial.suggest_float('ab_learning_rate', 0.01, 3.0),
        algorithm=trial.suggest_categorical('ab_algorithm', ['SAMME', 'SAMME.R'])
    )

def suggest_model(trial, model_type):
    if model_type == ModelType.NearestNeighbours.name:
        return suggest_nearest_neighbours_model(trial)
    elif model_type == ModelType.SupportVectorMachine.name:
        return suggest_svm_model(trial)
    elif model_type == ModelType.BayessianClassifier.name:
        return suggest_baessian_classifier(trial)
    elif model_type == ModelType.DecisionTree.name:
        return suggest_decision_tree(trial)
    elif model_type == ModelType.AdaBoost.name:
        return suggest_adaboost_model(trial)
    else:
        raise RuntimeError('Unexpected model type in suggest_model: {}'.format(model_type))

In [6]:
def objective(trial):
    model_type = trial.suggest_categorical('model_type', ModelType._member_names_)
    print(model_type)
    model = suggest_model(trial=trial, model_type=model_type)
    return cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=5, n_jobs=-1).mean()

In [7]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2023-06-08 14:32:33,406] A new study created in memory with name: no-name-e3ac0f5e-7ed0-4de3-aa28-37eb0e6c5f54


AdaBoost


[I 2023-06-08 14:32:37,379] Trial 0 finished with value: 0.9960697197539303 and parameters: {'model_type': 'AdaBoost', 'ab_n_estimators': 65, 'ab_learning_rate': 2.219961331232467, 'ab_algorithm': 'SAMME'}. Best is trial 0 with value: 0.9960697197539303.


SupportVectorMachine


[I 2023-06-08 14:32:39,262] Trial 1 finished with value: 0.5 and parameters: {'model_type': 'SupportVectorMachine', 'svm_gamma_cat': 'float', 'svm_gamma_num': 3.4588458636283246, 'svm_kernel': 'sigmoid', 'svm_coef0': 42.801329813290295, 'svm_shrinking': False, 'svm_probability': False, 'svm_tol': 0.5063563107685195, 'svm_C': 0.16914141386466067}. Best is trial 0 with value: 0.9960697197539303.
[I 2023-06-08 14:32:39,350] Trial 2 finished with value: 1.0 and parameters: {'model_type': 'SupportVectorMachine', 'svm_gamma_cat': 'scale', 'svm_kernel': 'rbf', 'svm_shrinking': False, 'svm_probability': True, 'svm_tol': 0.5265001843502919, 'svm_C': 3.7597516695928137}. Best is trial 2 with value: 1.0.


...

AdaBoost
SupportVectorMachine
SupportVectorMachine
SupportVectorMachine


[I 2023-06-08 14:32:44,758] Trial 93 finished with value: 1.0 and parameters: {'model_type': 'SupportVectorMachine', 'svm_gamma_cat': 'scale', 'svm_kernel': 'poly', 'svm_coef0': 36.522877124424, 'svm_degree': 4, 'svm_shrinking': True, 'svm_probability': True, 'svm_tol': 0.6542426653390289, 'svm_C': 1.8811473303083064}. Best is trial 2 with value: 1.0.
[I 2023-06-08 14:32:44,823] Trial 94 finished with value: 1.0 and parameters: {'model_type': 'SupportVectorMachine', 'svm_gamma_cat': 'scale', 'svm_kernel': 'rbf', 'svm_shrinking': True, 'svm_probability': False, 'svm_tol': 0.6844015075595455, 'svm_C': 2.2772144546927655}. Best is trial 2 with value: 1.0.
[I 2023-06-08 14:32:44,906] Trial 95 finished with value: 1.0 and parameters: {'model_type': 'SupportVectorMachine', 'svm_gamma_cat': 'float', 'svm_gamma_num': 4.7297253091814015, 'svm_kernel': 'poly', 'svm_coef0': 44.065801868912516, 'svm_degree': 1, 'svm_shrinking': False, 'svm_probability': True, 'svm_tol': 0.2252892574653515, 'svm_C'

SupportVectorMachine
SupportVectorMachine
SupportVectorMachine


[I 2023-06-08 14:32:44,979] Trial 96 finished with value: 0.9997265892002734 and parameters: {'model_type': 'SupportVectorMachine', 'svm_gamma_cat': 'auto', 'svm_kernel': 'rbf', 'svm_shrinking': True, 'svm_probability': False, 'svm_tol': 0.13501915914197593, 'svm_C': 3.065612818216401}. Best is trial 2 with value: 1.0.
[I 2023-06-08 14:32:45,011] Trial 97 finished with value: 1.0 and parameters: {'model_type': 'BayessianClassifier', 'bc_guess_priors': False, 'bc_var_smoothing': 0.0058580545862146225}. Best is trial 2 with value: 1.0.
[I 2023-06-08 14:32:45,063] Trial 98 finished with value: 1.0 and parameters: {'model_type': 'NearestNeighbours', 'knn_algorithm': 'brute', 'knn_leaf_size': 33, 'knn_n_neighbors': 222, 'knn_weights': 'uniform', 'knn_p': 2}. Best is trial 2 with value: 1.0.
[I 2023-06-08 14:32:45,107] Trial 99 finished with value: 1.0 and parameters: {'model_type': 'SupportVectorMachine', 'svm_gamma_cat': 'scale', 'svm_kernel': 'linear', 'svm_shrinking': False, 'svm_probabi

BayessianClassifier
NearestNeighbours
SupportVectorMachine


In [8]:
print(study.best_params)

{'model_type': 'SupportVectorMachine', 'svm_gamma_cat': 'scale', 'svm_kernel': 'rbf', 'svm_shrinking': False, 'svm_probability': True, 'svm_tol': 0.5265001843502919, 'svm_C': 3.7597516695928137}


In [9]:
optuna.visualization.plot_optimization_history(study)

In [10]:
optuna.visualization.plot_param_importances(study, params=list(study.best_params.keys()))

In [11]:
random_sampler_study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=1337))
random_sampler_study.optimize(objective, n_trials=100)

[I 2023-06-08 14:32:55,805] A new study created in memory with name: no-name-7cc2e979-5b93-4e00-a8bd-52cf0e5f9988
[I 2023-06-08 14:32:55,846] Trial 0 finished with value: 0.9960697197539303 and parameters: {'model_type': 'DecisionTree', 'dt_limit_max_depth': True, 'dt_max_depth': 98, 'dt_limit_max_leaf_nodes': True, 'dt_max_leaf_nodes': 119, 'dt_splitter': 'best', 'dt_criterion': 'gini', 'dt_min_samples_split': 40, 'dt_min_saples_leaf': 19, 'dt_max_features': None, 'dt_min_impurity_decrease': 0.09390420271320782, 'dt_ccp_alpha': 0.2881671478120357}. Best is trial 0 with value: 0.9960697197539303.
[I 2023-06-08 14:32:55,891] Trial 1 finished with value: 1.0 and parameters: {'model_type': 'NearestNeighbours', 'knn_algorithm': 'auto', 'knn_leaf_size': 86, 'knn_n_neighbors': 202, 'knn_weights': 'uniform', 'knn_p': 3}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:55,937] Trial 2 finished with value: 1.0 and parameters: {'model_type': 'NearestNeighbours', 'knn_algorithm': 'auto', 'kn

DecisionTree
NearestNeighbours
NearestNeighbours
BayessianClassifier
AdaBoost
SupportVectorMachine


[I 2023-06-08 14:32:56,053] Trial 5 finished with value: 1.0 and parameters: {'model_type': 'SupportVectorMachine', 'svm_gamma_cat': 'scale', 'svm_kernel': 'poly', 'svm_coef0': 36.2250001095229, 'svm_degree': 4, 'svm_shrinking': False, 'svm_probability': False, 'svm_tol': 0.4202663081521921, 'svm_C': 2.390828886990141}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:56,090] Trial 6 finished with value: 0.9960697197539303 and parameters: {'model_type': 'AdaBoost', 'ab_n_estimators': 58, 'ab_learning_rate': 2.2744810713741876, 'ab_algorithm': 'SAMME.R'}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:56,137] Trial 7 finished with value: 0.5 and parameters: {'model_type': 'DecisionTree', 'dt_limit_max_depth': False, 'dt_limit_max_leaf_nodes': True, 'dt_max_leaf_nodes': 125, 'dt_splitter': 'random', 'dt_criterion': 'log_loss', 'dt_min_samples_split': 19, 'dt_min_saples_leaf': 2, 'dt_max_features': 'log2', 'dt_min_impurity_decrease': 0.4575127023086554, 'dt_ccp_alpha': 0.7255666

AdaBoost
DecisionTree
NearestNeighbours
DecisionTree
BayessianClassifier


[I 2023-06-08 14:32:56,293] Trial 10 finished with value: 1.0 and parameters: {'model_type': 'BayessianClassifier', 'bc_guess_priors': True, 'bc_priors': 0.2624024528483213, 'bc_var_smoothing': 0.007191570734054847}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:56,332] Trial 11 finished with value: 0.9960697197539303 and parameters: {'model_type': 'AdaBoost', 'ab_n_estimators': 32, 'ab_learning_rate': 1.3331895343889586, 'ab_algorithm': 'SAMME.R'}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:56,363] Trial 12 finished with value: 1.0 and parameters: {'model_type': 'SupportVectorMachine', 'svm_gamma_cat': 'auto', 'svm_kernel': 'poly', 'svm_coef0': 31.49656550720801, 'svm_degree': 1, 'svm_shrinking': False, 'svm_probability': True, 'svm_tol': 0.19669920749320105, 'svm_C': 1.4315476452025202}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:56,387] Trial 13 finished with value: 1.0 and parameters: {'model_type': 'BayessianClassifier', 'bc_guess_priors': False, 'bc_var

...

SupportVectorMachine
BayessianClassifier
BayessianClassifier
NearestNeighbours
NearestNeighbours


[I 2023-06-08 14:32:59,558] Trial 88 finished with value: 1.0 and parameters: {'model_type': 'NearestNeighbours', 'knn_algorithm': 'auto', 'knn_leaf_size': 101, 'knn_n_neighbors': 180, 'knn_weights': 'distance', 'knn_p': 3}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:59,641] Trial 89 finished with value: 0.9960697197539303 and parameters: {'model_type': 'DecisionTree', 'dt_limit_max_depth': False, 'dt_limit_max_leaf_nodes': True, 'dt_max_leaf_nodes': 286, 'dt_splitter': 'best', 'dt_criterion': 'log_loss', 'dt_min_samples_split': 50, 'dt_min_saples_leaf': 29, 'dt_max_features': None, 'dt_min_impurity_decrease': 0.47681097389582583, 'dt_ccp_alpha': 0.695090184475064}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:59,674] Trial 90 finished with value: 0.9960697197539303 and parameters: {'model_type': 'AdaBoost', 'ab_n_estimators': 87, 'ab_learning_rate': 1.2284616165500015, 'ab_algorithm': 'SAMME.R'}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:59,711] Trial 91 f

DecisionTree
AdaBoost
NearestNeighbours
AdaBoost
NearestNeighbours
NearestNeighbours


[I 2023-06-08 14:32:59,820] Trial 94 finished with value: 1.0 and parameters: {'model_type': 'NearestNeighbours', 'knn_algorithm': 'ball_tree', 'knn_leaf_size': 126, 'knn_n_neighbors': 392, 'knn_weights': 'distance', 'knn_p': 1}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:59,878] Trial 95 finished with value: 0.9569719753930281 and parameters: {'model_type': 'NearestNeighbours', 'knn_algorithm': 'ball_tree', 'knn_leaf_size': 101, 'knn_n_neighbors': 356, 'knn_weights': 'uniform', 'knn_p': 3}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:59,910] Trial 96 finished with value: 0.9960697197539303 and parameters: {'model_type': 'AdaBoost', 'ab_n_estimators': 88, 'ab_learning_rate': 2.0437123319284587, 'ab_algorithm': 'SAMME'}. Best is trial 1 with value: 1.0.
[I 2023-06-08 14:32:59,943] Trial 97 finished with value: 0.9960697197539303 and parameters: {'model_type': 'AdaBoost', 'ab_n_estimators': 56, 'ab_learning_rate': 2.921766714815733, 'ab_algorithm': 'SAMME.R'}. Best is 

NearestNeighbours
AdaBoost
AdaBoost
AdaBoost
SupportVectorMachine


In [12]:
print(random_sampler_study.best_params)

{'model_type': 'NearestNeighbours', 'knn_algorithm': 'auto', 'knn_leaf_size': 86, 'knn_n_neighbors': 202, 'knn_weights': 'uniform', 'knn_p': 3}


In [13]:
optuna.visualization.plot_optimization_history(random_sampler_study)

In [14]:
optuna.visualization.plot_param_importances(random_sampler_study, params=list(study.best_params.keys()))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df, target, train_size=train_size, shuffle=True)
study_best_model = KNeighborsClassifier(algorithm='kd_tree', n_neighbors=339, weights='uniform', leaf_size=114, p=1)
study_best_model.fit(X_train, y_train)
prediction = study_best_model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, prediction)

random_sampler_study_best_model = KNeighborsClassifier(algorithm='auto', n_neighbors=213, weights='distance', p=1)
random_sampler_study_best_model.fit(X_train, y_train)
random_sampler_prediction = random_sampler_study_best_model.predict_proba(X_test)[:, 1]
random_sampler_roc_auc = roc_auc_score(y_test, random_sampler_prediction)

print('Study best model roc auc score: {}'.format(roc_auc))
print('Random study best model roc auc score: {}'.format(random_sampler_roc_auc))

Study best model roc auc score: 0.9999849138581299
Random study best model roc auc score: 1.0
